### LINEAMIENTOS

La inmobiliaria Properati publica periódicamente información sobre ofertas de propiedades para venta y alquiler. 
Ud. deberá asesorar a la inmobiliaria a desarrollar un modelo de regresión que permita predecir el precio por metro 
cuadrado de una propiedad. El objetivo final, a concretarse en el Desafío 2, es que el modelo que desarrollen sea utilizado 
como tasador automático a ser aplicados a las próximas propiedades que sean comercializadas por la empresa. 
Para ello, la empresa le provee de un dataset correspondiente al primer semestre de 2017.

El dataset es de tamaño entre pequeño y mediano, pero tiene dos complejidades a las que deberá prestarle atención:

● Peso de missing data en algunas variables relevantes.
● Será importante tener en cuenta el problema de la influencia espacial en los precios por metro cuadrado. 
En efecto, es probable que existan diferencias importantes de en las diferentes geografías, barrios y zonas analizadas.

Objetivos:
● Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos 
perdidos en ciertas variables.
● Realizar un análisis descriptivo de las principales variables.
● Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.

Datos: Descargar el dataset de: https://drive.google.com/file/d/0BzVrTKc02N8qNUdDSExBQlFTNlU/view

Requisitos y material a entregar
La limpieza y el análisis de los datos deberán ser entregados en una Jupyter notebook que satisfaga los 
requerimientos del proyecto. La notebook deberá estar debidamente comentada. Además, los grupos deberán crear 
un repositorio para el proyecto en Github.

Para la presentación en clase se deben armar algunos slides no técnicos para exponer en no más de 10 minutos 
(PPT o Google Slides). La presentación debe constar de una introducción (planteo del problema, la pregunta, 
la descripción del dataset, etc.), un desarrollo de los análisis realizados (análisis descriptivo, análisis de correlaciones 
preliminares, visualizaciones preliminares) y una exposición de los principales resultados y conclusiones.

Fecha de entrega
● El material deberá entregarse en la clase 17 del curso, día viernes 13 de septiembre.

Dataset

El dataset contiene información sobre todas las propiedades georeferenciadas de la base de datos de la empresa. 
La información de cada propiedad que incluye es la siguiente:

● Fecha de creación
● Tipo de la propiedad (house, apartment, ph)
● Operación del aviso (sell, rent)
● Nombre del lugar
● Nombre del lugar + nombre de sus ‘padres’
● ID de geonames del lugar (si está disponible)
● Latitud, longitud
● Precio original del aviso
● Moneda original del aviso (ARS, USD)
● Precio del aviso en moneda local (ARS)
● Precio aproximado en USD
● Superficie en m2
● Superficie cubierta en m2
● Precio en USD/m2
● Precio por m2
● N° de piso, si corresponde
● Ambientes
● URL en Properati
● Descripción
● Título
● URL de un thumbnail de la primer foto

¿Cómo empezar? Sugerencias

Agreguen toda otra información construida a partir de los datos originales (o incluso información externa) que consideren 
relevante y útil para resolver los objetivos planteados.

Aprovechen las herramientas de Pandas: groupby, summation, pivot_tables y otras aplicaciones y métodos de los 
DataFrames hacen mucho más simples los cálculos y otras agregaciones de los datos.

En la presentación de los resultados tengan en cuenta que es altamente probable que la audiencia no tenga un nivel técnico, 
así que mantengan el lenguaje en un nivel accesible.

En términos generales, recuerden las siguientes sugerencias:

● Escribir un pseudocódigo antes de empezar a codear. Suele ser muy útil para darle un esquema y una lógica generales 
al análisis.

● Leer la documentación de cualquier tecnología o herramienta de análisis que usen. A veces no hay tutoriales para todo 
y los documentos y las ayudas son fundamentales para entender el funcionamiento de las herramientas utilizadas.

● Documentar todos los pasos, transformaciones, comandos y análisis que realicen.

Recursos útiles

● Documentación de la librería GeoPandas
● Github

Acción: Importo file y realizo sample para tener una primera idea de algunas columnas y sus datos

In [3]:
import pandas as pd
base = pd.read_csv(r"C:\Users\GuillermoPortero\Desktop\Digital House\Desafio\properatti.csv")
base.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
92302,92302,sell,apartment,Villa Crespo,|Argentina|Capital Federal|Villa Crespo|,Argentina,Capital Federal,3427458.0,"-34.597985113,-58.426512964",-34.597985,...,100.0,2928.575758,3865.720000,NaN,NaN,NaN,http://www.properati.com.ar/1bcnh_venta_depart...,Este emprendimiento está ubicado en pleno barr...,Departamento de tres ambientes con terraza,https://thumbs4.properati.com/1/AMyICsG3VZBGx2...
65221,65221,sell,house,San Isidro,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|,Argentina,Bs.As. G.B.A. Zona Norte,3428983.0,NaN,NaN,...,504.0,1269.841270,1269.841270,NaN,NaN,NaN,http://www.properati.com.ar/1abyz_venta_casa_s...,MUY BUENA CASA SOBRE LOTE DOBLE!!PB: Hall de e...,Muy buena casa en Barrio Cerrado Camino Real,https://thumbs4.properati.com/0/3FhiuGTDxY6jfq...
54707,54707,sell,house,Villa Carlos Paz,|Argentina|Córdoba|Villa Carlos Paz|,Argentina,Córdoba,3832791.0,"-31.420572,-64.49949",-31.420572,...,90.0,294.642857,1833.333333,NaN,NaN,NaN,http://www.properati.com.ar/19no7_venta_casa_v...,CÓD: CCP177Casa ubicada en Barrio La CuestaCon...,CCP177- Hermosa propiedad de 2 dormitorios más...,https://thumbs4.properati.com/3/l98V76OBlCfX0T...
11613,11613,sell,house,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-38.0054771,-57.5426106",-38.005477,...,272.0,711.111111,1176.470588,NaN,5.0,NaN,http://www.properati.com.ar/16c5l_venta_casa_m...,"Excelente propiedad en Muy buena ubicación, co...",Excelente propiedad,https://thumbs4.properati.com/1/d_lfeVfgWvkHyR...
57577,57577,sell,house,Barrio Los Alisos,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.39092961,-58.65023804",-34.390930,...,286.0,1982.758621,2412.587413,NaN,NaN,NaN,http://www.properati.com.ar/19tn1_venta_casa_b...,Barrio Naútico con salida al río y amarra prop...,NORDELTA - LAS CALETAS 100,https://thumbs4.properati.com/5/OYeKvhorSeRfT2...


Acción: Verifico cómo está conformada la estructura del file.

In [5]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

Observaciones: 
1) Es un Dataframe
2) Son 121220 filas y 26 columnas
3) A priori se observan Labels poco legibles ("Unnamed: 0")
4) Hay poca información para ciertos campos ("lat-lon","expenses","rooms")
5) Habría campos con info repetida ("lat-lon" vs "lat" y "lon"), 6. Hay columnas no relevantes("image...","...url")

Acción: Recorro campo a campo y cuento la cantidad de valores diferentes

In [7]:
for i in base.columns:
    print("\n'{}' values:\n".format(i),base[i].value_counts())


'Unnamed: 0' values:
 2047     1
54576    1
29988    1
32037    1
25894    1
27943    1
5416     1
7465     1
1322     1
3371     1
13612    1
15661    1
9518     1
11567    1
56625    1
17698    1
50482    1
52531    1
62772    1
64821    1
58678    1
60727    1
38200    1
40249    1
34106    1
36155    1
46396    1
48445    1
19747    1
23841    1
        ..
29403    1
31450    1
25305    1
27352    1
86655    1
43712    1
41665    1
47810    1
45763    1
35524    1
33477    1
39622    1
37575    1
60104    1
58057    1
64202    1
62155    1
51916    1
49869    1
56014    1
53967    1
10960    1
8913     1
15058    1
13011    1
2772     1
725      1
6870     1
4823     1
0        1
Name: Unnamed: 0, Length: 121220, dtype: int64

'operation' values:
 sell    121220
Name: operation, dtype: int64

'property_type' values:
 apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64

'place_name' values:
 Córdoba                          


'surface_covered_in_m2' values:
 40.0       2736
50.0       2184
45.0       2142
60.0       1888
100.0      1848
70.0       1724
200.0      1717
30.0       1653
42.0       1640
80.0       1512
120.0      1498
35.0       1459
90.0       1337
20.0       1313
48.0       1276
150.0      1257
65.0       1245
38.0       1242
55.0       1229
43.0       1205
47.0       1190
46.0       1134
75.0       1120
32.0       1082
36.0       1052
44.0       1048
180.0      1011
110.0       997
37.0        989
49.0        976
           ... 
3895.0        1
9435.0        1
614.0         1
35000.0       1
7788.0        1
1630.0        1
1132.0        1
1130.0        1
910.0         1
1750.0        1
1880.0        1
761.0         1
759.0         1
1710.0        1
895.0         1
7000.0        1
38240.0       1
543.0         1
1428.0        1
607.0         1
810.0         1
506.0         1
4350.0        1
1661.0        1
3100.0        1
944.0         1
1589.0        1
628.0         1
849.0         1
2300.0


'description' values:
 AVISO LEGAL: Las descripciones arquitectónicas y funcionales, valores de expensas, impuestos y servicios, fotos y medidas de este inmueble son aproximados. Los datos fueron proporcionados por el propietario y pueden no estar actualizados a la hora de la visualización de este aviso por lo cual pueden arrojar inexactitudes y discordancias con las que surgen de los las facturas, títulos y planos legales del inmueble. Los precios indicados podrán modificarse sin previo aviso. La fechas de inicio de obra o posesión son estimadas, podrán ser reprogramadas por la Dirección de obra y dependerán a su vez de un proceso de aprobaciones municipales u otros organismos intervinientes. El interesado deberá realizar las verificaciones respectivas previamente a la realización de cualquier operación, requiriendo por sí o sus profesionales las copias necesarias de la documentación que corresponda. Venta supeditada al cumplimiento por parte del propietario de los requisitos de la r

Observaciones:
#_Unnamed: tiene 121220 valores únicos del tipo integer. Parecería un tipo de índice. A priori no agregaría valor al estimador
#_operation: tiene 121220 valores iguales el cual es "sell". Campo sería descartable.  
#_property_type: tiene 4 valores diferentes (apartment,house,PH,store). Falta corroborar si faltan valores (y cuantos)
#_place_name: tiene muchos valores diferentes que refieren a Localidades, Barrios. Falta corroborar si faltan valores (y cuantos)
#_place_with_parent_names: jerarquía padre/hijo/nieto... Verificar si sirve para complementar info faltante de "place_name" (si faltara) o si no hay concordancia entre dichos campos. Serviría para tener un valor estimado por Provincia, Localidad, Zona
#_country_name: tiene 121220 valores iguales el cual es "Argentina". Campo que no agrega valor.
#_state_name: tiene muchos valores diferentes que refieren Provincia. Falta corroborar si faltan valores (y cuantos). Serviría como base para estimar valores por provincia, y complementarse con place_with_parent_names en aquellos registros que le falte dato.
#_geonames_id: 646 valoes. A priori no agregaría valor al estimador. Utilizaría "lat-lon" para complementar "place_name"
#_'lat-lon': podría complementar a "place_name"
#_lat: a priori sus valores están incluidos en "lat_lon". El campo sería descartable
#_lon: a priori sus valores están incluidos en "lat_lon". El campo sería descartable
#_price: solo 9746 registros con valores. Se requiere de "currency" para interpretarlo. Corroborar si sirve para complementar otros price_usd_per_m2 .
#_currency: ~90k valores útiles. filtar registros que no estén en ARS o USD. Crear nueva columna llevando todo a USD (verificar si existe TC y sino importarlo para usarlo como DICC). 
#_price_aprox_local_currency: ~10.3k valores. Complementa aquellos casos sin "price_usd_per_m2","price", ni "price_aprox_usd" (si es que priorizo estimar en USD frente a ARS)
#_price_aprox_usd: ~10.3k valores. Complementa aquellos casos sin "price_usd_per_m2", ni "price"
#_surface_total_in_m2: ~1.6k valores
#_surface_covered_in_m2: ~1k valores. Principal complementador del precio en dólares para llegar a "price_usd_per_m2"
#_price_usd_per_m2: ~25.6k valores. Principal estimador de precios
#_price_per_m2: ~25k valores. Complementa "price_usd_per_m2" 
#_floor: ~200 valores. A priori no agregaría valor al estimador
#_rooms: calcular cantidad de valores. A priori no agregaría valor al estimador
#_expenses: ~1k. A priori no agregaría valor al estimador
#_properati_url:tiene 121220 valores únicos. Útil para detectar duplicados. Luego descartable
#_description: Campo sería descartable. 
#_title: ~73k. Se podría utilizar para complementar "property_type" o "place_name" ya que varios registros son del tipo "Departamento - Villa Urquiza"
#image_thumbnail: Campo sería descartable.

